<a href="https://colab.research.google.com/github/changsin/ModelTrain/blob/main/models/notebooks/voice-recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Voice recognition


In [1]:
!git clone https://github.com/changsin/ModelTrain.git
%cd ModelTrain/voice-recognition/

%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.13.1+cu116 (Tesla T4)


In [20]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=cd43f15161365bbe5d1c1be778d1925d98ebe58e5d285fbb9f9344aa3ee67927
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [73]:
%cd /content/ModelTrain/models/voice-recognition

import gc
import os
import pickle
from glob import glob
from pathlib import Path

import pandas as pd
import torch
import wandb

from torch import nn
from torch.utils.data import DataLoader

from data import CustomTokenizer, CustomDataset
from stt_model import Transformer

max_length = 30
batch_size = 32
num_layers = 6
d_model = 512
dff = 2048
num_heads = 8
dropout_rate = 0.1
learning_rate = 5e-5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
max_vocab_size = 5000

def bind_model(model, parser):
    # 학습한 모델을 저장하는 함수입니다.
    def save(dir_name, *parser):
        # directory
        os.makedirs(dir_name, exist_ok=True)
        save_dir = os.path.join(dir_name, 'checkpoint')
        save_checkpoint(dict_for_infer, save_dir)

        with open(os.path.join(dir_name, "dict_for_infer"), "wb") as f:
            pickle.dump(dict_for_infer, f)

        print("저장 완료!")

    # 저장한 모델을 불러올 수 있는 함수입니다.
    def load(dir_name, *parser):
        save_dir = os.path.join(dir_name, 'checkpoint')

        global checkpoint
        checkpoint = torch.load(save_dir)

        model.load_state_dict(checkpoint['model'])

        global dict_for_infer
        with open(os.path.join(dir_name, "dict_for_infer"), 'rb') as f:
            dict_for_infer = pickle.load(f)

        print("로딩 완료!")

    # def infer(test_path, **kwparser):
    #     device = checkpoint['device']
    #     test_file_list = path_loader(test_path, is_test=True)
    #     test_dataset = CustomDataset(test_file_list, None, 160000, 'test')
    #     test_data_loader = DataLoader(test_dataset,
    #                                   batch_size=10)
    #     result_list = []

    #     for step, batch in enumerate(test_data_loader):
    #         inp = batch['magnitude'].to(device)
    #         output, _ = evaluate(model, inp)
    #         result_list.extend(output)

    #     prob = [1] * len(result_list)

    #     # DONOTCHANGE: They are reserved for nsml
    #     # 리턴 결과는 [(확률, 0 or 1)] 의 형태로 보내야만 리더보드에 올릴 수 있습니다. 리더보드 결과에 확률의 값은 영향을 미치지 않습니다
    #     # return list(zip(pred.flatten(), clipped.flatten()))
    #     return list(zip(prob, result_list))

    # # DONOTCHANGE: They are reserved for nsml
    # # nsml에서 지정한 함수에 접근할 수 있도록 하는 함수입니다.
    # nsml.bind(save=save, load=load, infer=infer)


def evaluate(model, imgs):
    model.to(device)
    # as the target is english, the first word to the transformer should be the
    # english start token.
    tokenizer = dict_for_infer['tokenizer']
    decoder_input = torch.tensor([tokenizer.txt2idx['<sos>']] * imgs.size(0), dtype=torch.long).to(device)
    output = decoder_input.unsqueeze(1).to(device)
    enc_output = None
    for i in range(max_length + 1):
        # predictions.shape == (batch_size, seq_len, vocab_size)
        with torch.no_grad():
            # predictions, attention_weights, enc_output = transformer([imgs, output, enc_output])
            predictions, attention_weights, enc_output = model([imgs, output, enc_output])
        # select the last token from the seq_len dimension
        predictions_ = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = torch.tensor(torch.argmax(predictions_, axis=-1), dtype=torch.int32)

        output = torch.cat([output, predicted_id], dim=-1)
    output = output.cpu().numpy()

    result_list = []
    token_list = []
    for token in output:
        summary = tokenizer.convert(token)
        result_list.append(summary)
        token_list.append(token)

    return result_list, token_list


def train_step(batch_item, is_training):
    src = batch_item['magnitude'].to(device)
    tar = batch_item['target'].to(device)
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    print("batch_item: {}".format(type(batch_item)))
    print("src.shape: {} tar.shape: {}".format(src.shape, tar.shape))
    print("tar_inp.shape: {} tar_real.shape: {}".format(tar_inp.shape,
                                                        tar_real.shape))

    if is_training:
        # transformer.train()
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output, _, _ = model([src, tar_inp, None])
            # output, _, _ = transformer([src, tar_inp, None])
            loss = loss_function(tar_real, output)
        acc = accuracy_function(tar_real, output)
        loss.backward()
        optimizer.step()
        lr = optimizer.param_groups[0]["lr"]
        return loss, acc, round(lr, 10)
    else:
        # transformer.eval()
        model.eval()
        with torch.no_grad():
            output, _, _ = model([src, tar_inp, None])
            # output, _, _ = transformer([src, tar_inp, None])
            loss = loss_function(tar_real, output)
        acc = accuracy_function(tar_real, output)
        return loss, acc


def loss_function(real, pred):
    mask = torch.logical_not(torch.eq(real, 0))
    loss_ = criterion(pred.permute(0, 2, 1), real)
    mask = torch.tensor(mask, dtype=loss_.dtype)
    loss_ = mask * loss_

    return torch.sum(loss_) / torch.sum(mask)


def accuracy_function(real, pred):
    accuracies = torch.eq(real, torch.argmax(pred, dim=2))
    mask = torch.logical_not(torch.eq(real, 0))
    accuracies = torch.logical_and(mask, accuracies)
    accuracies = torch.tensor(accuracies, dtype=torch.float32)
    mask = torch.tensor(mask, dtype=torch.float32)

    return torch.sum(accuracies) / torch.sum(mask)

def glob_files(folder, file_type='*'):
    search_string = os.path.join(folder, file_type)
    files = glob(search_string)

    print('Searching files ', search_string)
    paths = []
    for f in files:
      if os.path.isdir(f):
        sub_paths = glob_files(f + '/')
        paths += sub_paths
      else:
        paths.append(f)

    # We sort the images in alphabetical order to match them
    #  to the annotation files
    paths.sort()

    return paths


def glob_folders(folder, file_type='*'):
    search_string = os.path.join(folder, file_type)
    files = glob(search_string)

    print('Searching folders {} {}'.format(search_string, files))
    paths = []
    for f in files:
      if os.path.isdir(f):
        paths.append(f)

    # We sort the images in alphabetical order to match them
    #  to the annotation files
    paths.sort()

    return paths


def glob_files_all(folder, file_type='*'):
    print("Searching in {} from {}".format(folder, os.getcwd()))
    sub_folders = glob_folders(folder)
    print("Found {} sub folders".format(len(sub_folders)))

    files = []
    for sub_folder in sub_folders:
        tmp_files = glob_files(sub_folder, file_type)
        if tmp_files:
            files.extend(tmp_files)
    print("Found {} files".format(len(files)))
    return files

def path_loader(root_path, divide_id=8000, use_column=1, is_training=True):
    print("root_path: {} divide_id: {} use_column: {} is_test: {}".format(root_path, divide_id, use_column, is_training))

    if is_training:
        train_path = root_path
        # os.path.join(root_path, 'train')
        file_list = sorted(glob_files_all(os.path.join(train_path, 'train_data', '')))
        # file_list = sorted(glob_files_all(os.path.join(train_path, 'train_lite', '')))
        print("Data files loaded {}".format(len(file_list)))
        # file_list = sorted(glob(os.path.join(train_path, 'train_data', '*')))
        # label = pd.read_csv(os.path.join(train_path, 'labels_tw_lite.txt'))
        label = pd.read_csv(os.path.join(train_path, 'train_label.txt'))
        # label = pd.read_csv(os.path.join(train_path, 'labels_ai-hub_tw_shuffle.txt'))
        print("Loaded label {}".format(len(label)))

        file_dict = dict()
        for full_file_path in file_list:
            filename = Path(os.path.basename(full_file_path)).stem
            if file_dict.get(filename.lower()):
                print("ERROR: duplicate file name found {}".format(filename))
            else:
                file_dict[filename.lower()] = full_file_path

        print("file_dict {}".format(len(file_dict)))
        for id, (key, val) in enumerate(file_dict.items()):
            print("{} {}".format(key, val))
            if id > 3:
                break

        file_list_selected = []
        data_file_paths = label.iloc[:, 0]
        for data_file_path in data_file_paths:
            data_filename = Path(os.path.basename(data_file_path)).stem
            data_filename = data_filename.replace("[\"", "")
            if file_dict.get(data_filename.lower()):
                file_list_selected.append(file_dict[data_filename.lower()])
            else:
                print("ERROR: file not found {}".format(data_filename))

        return file_list_selected[:divide_id], label[:divide_id]

    else:
        return sorted(glob(os.path.join(root_path, 'test_data', '*')))

def save_checkpoint(checkpoint, dir):
    torch.save(checkpoint, os.path.join(dir))

/content/ModelTrain/models/voice-recognition


In [74]:
epochs = 5
divide_id = 100
use_column = 1

DATASET_PATH = "/content/ModelTrain/data/senior_voice_commands/train"
file_list, label = path_loader(DATASET_PATH, divide_id, use_column)

print("Loaded files {} labels {}".format(len(file_list), len(label)))

split_num = int(len(label) * 0.9)
train_file_list = file_list[:split_num]
val_file_list = file_list[split_num:]

train_label = label.iloc[:split_num]
val_label = label.iloc[split_num:]

tokenizer = CustomTokenizer(max_length=max_length, max_vocab_size=max_vocab_size)
tokenizer.fit(train_label.iloc[:, 1])

target_size = len(tokenizer.txt2idx)

train_tokens = tokenizer.txt2token(train_label.iloc[:, 1])
val_tokens = tokenizer.txt2token(val_label.iloc[:, 1])
train_dataset = CustomDataset(train_file_list, train_tokens)
valid_dataset = CustomDataset(val_file_list, val_tokens)

print("train_tokens: {} train_file_list: {}".format(len(train_tokens), len(train_file_list)))
print("val_tokens: {} val_file_list: {}".format(len(val_tokens), len(val_file_list)))

print("train_token samples: {} train_file samples: {}".format(train_label.iloc[:, 1][:3], train_file_list[:3]))
print("val_token samples: {} val_file samples: {}".format(val_label.iloc[:, 1][:3], val_file_list[:3]))
train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True)

valid_dataloader = DataLoader(valid_dataset,
                              batch_size=batch_size,
                              shuffle=False)


root_path: /content/ModelTrain/data/senior_voice_commands/train divide_id: 100 use_column: 1 is_test: True
Searching in /content/ModelTrain/data/senior_voice_commands/train/train_data/ from /content/ModelTrain/models/voice-recognition
Searching folders /content/ModelTrain/data/senior_voice_commands/train/train_data/* ['/content/ModelTrain/data/senior_voice_commands/train/train_data/o_0271']
Found 1 sub folders
Searching files  /content/ModelTrain/data/senior_voice_commands/train/train_data/o_0271/*
Found 100 files
Data files loaded 100
Loaded label 100
file_dict 100
o_0271-13003-02-01-kes-f-08-a /content/ModelTrain/data/senior_voice_commands/train/train_data/o_0271/o_0271-13003-02-01-KES-F-08-A.wav
o_0271-13004-02-01-kes-f-08-a /content/ModelTrain/data/senior_voice_commands/train/train_data/o_0271/o_0271-13004-02-01-KES-F-08-A.wav
o_0271-13005-02-01-kes-f-08-a /content/ModelTrain/data/senior_voice_commands/train/train_data/o_0271/o_0271-13005-02-01-KES-F-08-A.wav
o_0271-13006-02-01-kes

In [75]:
model = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    target_size=max_vocab_size + 4,
    pe_target=max_length + 1,
    device=device,
    rate=dropout_rate
)

# bind_model(model=model, parser=args)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

use_wandb = False
if use_wandb:
    wandb.init("voice-recognition")


for epoch in range(epochs):
    gc.collect()
    total_train_loss, total_valid_loss = 0, 0
    total_train_acc, total_valid_acc = 0, 0

    training = True
    avg_batch_loss, avg_batch_acc = 0, 0
    iterations = 0
    for batch in train_dataloader:
        batch = batch
        batch_loss, batch_acc, lr = train_step(batch, training)
        total_train_loss += batch_loss
        total_train_acc += batch_acc
        if use_wandb:
            wandb.log({"train_batch_acc": batch_acc, "train_batch_loss":batch_loss})

        avg_batch_loss += batch_loss
        avg_batch_acc += batch_acc
        iterations += 1

    if use_wandb:
        wandb.log({
            "avg_train_batch_acc": avg_batch_acc / float(iterations),
            "avg_train_batch_loss": avg_batch_loss / float(iterations)})

    print(f'avg_train_batch_acc: {avg_batch_acc / float(iterations)}')
    print(f'avg_train_batch_loss: {avg_batch_acc / float(iterations)}')

    training = False
    iterations = 0
    avg_batch_loss, avg_batch_acc = 0, 0
    for batch in valid_dataloader:
        batch = batch
        batch_loss, batch_acc = train_step(batch, training)
        total_valid_loss += batch_loss
        total_valid_acc += batch_acc
        if use_wandb:
            wandb.log({"valid_batch_acc": batch_acc, "valid_batch_loss": batch_loss})

        avg_batch_loss += batch_loss
        avg_batch_acc += batch_acc
        iterations += 1

    if use_wandb:
        wandb.log({"avg_valid_batch_acc": avg_batch_acc/float(iterations),
                    "avg_valid_batch_loss": avg_batch_loss/float(iterations)})

    print('=================Epoch: {} Iterations: {}'.format(epoch, iterations))
    print(f'avg_valid_batch_acc: {avg_batch_acc / float(iterations)}')
    print(f'avg_valid_batch_loss: {avg_batch_acc / float(iterations)}')

    print(f'total_train_loss: {total_train_loss}')
    print(f'total_valid_loss: {total_valid_loss}')
    print(f'total_train_acc : {total_train_acc}')
    print(f'total_valid_acc : {total_valid_acc}')

    dict_for_infer = {
        'model': model.state_dict(),
        'max_length': max_length,
        'target_size': target_size,
        'num_layers': num_layers,
        'd_model': d_model,
        'dff': dff,
        'num_heads': num_heads,
        'dropout_rate': dropout_rate,
        'epochs': epochs,
        'learning_rate': learning_rate,
        'tokenizer': tokenizer,
        'device': device
    }

    if use_wandb:
        wandb.log({
            "total_train_loss": total_train_loss, "total_valid_loss": total_valid_loss,
            "total_train_acc": total_train_acc,   "total_valid_acc": total_valid_acc
                })
    print("Writing to ./checkpoint_col{}_{}".format(use_column, epoch))
    save_checkpoint(checkpoint=dict_for_infer, dir='./checkpoint'.format(use_column))


batch_item: <class 'dict'>
src.shape: torch.Size([32, 3, 257, 313]) tar.shape: torch.Size([32, 32])
tar_inp.shape: torch.Size([32, 31]) tar_real.shape: torch.Size([32, 31])


/content/ModelTrain/models/voice-recognition/stt_model.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq = torch.tensor(torch.eq(seq, 0), dtype=torch.float32)
<ipython-input-73-5c2e079c9441>:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.tensor(mask, dtype=loss_.dtype)
<ipython-input-73-5c2e079c9441>:162: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  accuracies = torch.tensor(accuracies, dtype=torch.float32)
<ipython-input-73-5c2e079c9441>:163: UserWarning: To copy construct from a tensor, it is recommended to use 

batch_item: <class 'dict'>
src.shape: torch.Size([32, 3, 257, 313]) tar.shape: torch.Size([32, 32])
tar_inp.shape: torch.Size([32, 31]) tar_real.shape: torch.Size([32, 31])
batch_item: <class 'dict'>
src.shape: torch.Size([26, 3, 257, 313]) tar.shape: torch.Size([26, 32])
tar_inp.shape: torch.Size([26, 31]) tar_real.shape: torch.Size([26, 31])
avg_train_batch_acc: 0.0
avg_train_batch_loss: 0.0
batch_item: <class 'dict'>
src.shape: torch.Size([10, 3, 257, 313]) tar.shape: torch.Size([10, 32])
tar_inp.shape: torch.Size([10, 31]) tar_real.shape: torch.Size([10, 31])
=================Epoch: 0 Iterations: 1
avg_valid_batch_acc: 0.0
avg_valid_batch_loss: 0.0
total_train_loss: 21.892440795898438
total_valid_loss: 5.677505016326904
total_train_acc : 0.0
total_valid_acc : 0.0
Writing to ./checkpoint_col1_0
batch_item: <class 'dict'>
src.shape: torch.Size([32, 3, 257, 313]) tar.shape: torch.Size([32, 32])
tar_inp.shape: torch.Size([32, 31]) tar_real.shape: torch.Size([32, 31])
batch_item: <class

In [51]:
!pip install torchviz
from torchviz import make_dot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4151 sha256=2aaad1051039da01a6e808076ee17a8b7ef4874e299470dd587c506135704897
  Stored in directory: /root/.cache/pip/wheels/05/7d/1b/8306781244e42ede119edbb053bdcda1c1f424ca226165a417
Successfully built torchviz


In [62]:
!pip install torchview

from torchview import draw_graph
from torchvision.models import resnet18, GoogLeNet, densenet, vit_b_16

# model_graph = draw_graph(model, input_size=(1,3,224,224), expand_nested=True)
model_graph = draw_graph(model, input_size=(3, 257, 313), expand_nested=True)
model_graph.visual_graph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/content/ModelTrain/models/voice-recognition/stt_model.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq = torch.tensor(torch.eq(seq, 0), dtype=torch.float32)


RuntimeError: ignored

In [82]:
for batch in train_dataloader:
  print(batch['target'].shape)

torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([26, 32])


In [87]:
print(train_dataloader[0])

TypeError: ignored

In [57]:
print(model)

Transformer(
  (encoder): CNN_Encoder(
    (feature_extract_model): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(in

In [90]:
train_dataloader

In [76]:
batch_item = train_dataloader.dataset[0]
type(batch_item)

# src = batch_item['magnitude'].to(device)
# tar = batch_item['target'].to(device)
# tar_inp = tar[:, :-1]
# tar_real = tar[:, 1:]

# y = model([src, tar_inp, None])
# make_dot(y.mean(), params=dict(model.named_parameters()))

dict

In [69]:
tar.shape

torch.Size([32])

In [2]:
!pip install colabcode
from colabcode import ColabCode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 42.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.4/347.4 KB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 119.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 KB 590.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 

In [ ]:
ColabCode(port=10000, password="aiworks1!")

Code Server can be accessed on: NgrokTunnel: "https://d1b3-104-199-221-107.ngrok.io" -> "http://localhost:10000"
[2023-02-22T07:08:05.139Z] info  code-server 3.10.2 387b12ef4ca404ffd39d84834e1f0776e9e3c005
[2023-02-22T07:08:05.140Z] info  Using user-data-dir ~/.local/share/code-server
[2023-02-22T07:08:05.153Z] info  Using config file ~/.config/code-server/config.yaml
[2023-02-22T07:08:05.153Z] info  HTTP server listening on http://127.0.0.1:10000 
[2023-02-22T07:08:05.153Z] info    - Authentication is enabled
[2023-02-22T07:08:05.153Z] info      - Using password from $PASSWORD
[2023-02-22T07:08:05.153Z] info    - Not serving HTTPS 


In [ ]:
!pip install protobuf==3.20.*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached protobuf-3.20.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.12
    Uninstalling protobuf-4.21.12:
      Successfully uninstalled protobuf-4.21.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
# !git pull
# %pip install -qr requirements.txt  # install dependencies

!python main.py --epochs 10 --iteration 10

torch version:  1.13.1+cu116
Searching in ../data/senior_voice_commands/train/train_data/ from /content/ModelTrain/voice-recognition
Searching folders ../data/senior_voice_commands/train/train_data/* ['../data/senior_voice_commands/train/train_data/o_0271']
Found 1 sub folders
Searching files  ../data/senior_voice_commands/train/train_data/o_0271/*
Found 450 files
Data files loaded 450
Loaded label 450
train_tokens: 405 val_tokens: 45
train_file_list: 405 val_file_list: 45
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can al

In [ ]:
!python main.py --epochs 10 --iteration 100

torch version:  1.13.1+cu116
Searching in ../data/senior_voice_commands/train/train_data/ from /content/ModelTrain/voice-recognition
Searching folders ../data/senior_voice_commands/train/train_data/* ['../data/senior_voice_commands/train/train_data/o_0271']
Found 1 sub folders
Searching files  ../data/senior_voice_commands/train/train_data/o_0271/*
Found 450 files
Data files loaded 450
Loaded label 450
train_tokens: 405 val_tokens: 45
train_file_list: 405 val_file_list: 45
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can al